In [289]:
#import des fichiers et load des ressources
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

data = pd.read_csv('2016_Building_Energy_Benchmarking.csv')
pd.options.display.max_columns = data.shape[1]

In [290]:
data.sample(1)

,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,Address,City,State,ZipCode,TaxParcelIdentificationNumber,CouncilDistrictCode,Neighborhood,Latitude,Longitude,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),ListOfAllPropertyUseTypes,LargestPropertyUseType,LargestPropertyUseTypeGFA,SecondLargestPropertyUseType,SecondLargestPropertyUseTypeGFA,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,YearsENERGYSTARCertified,ENERGYSTARScore,SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SourceEUI(kBtu/sf),SourceEUIWN(kBtu/sf),SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),DefaultData,Comments,ComplianceStatus,Outlier,TotalGHGEmissions,GHGEmissionsIntensity
212,334,2016,NonResidential,Large Office,West 8th,2001 - 8th Avenue,Seattle,WA,98121.0,0659000750,7,DOWNTOWN,47.61578,-122.33726,2009,1.0,25,758043,228668,529375,"Office, Parking",Office,582725.0,Parking,191082.0,NaN,NaN,20162015,95.0,46.400002,46.400002,142.399994,142.399994,27018020.0,27018020.0,0.0,7658746.5,26131643.0,8863.726563,886373.0,False,NaN,Compliant,NaN,229.25,0.3


In [291]:
#variables cohérentes
cols = ['OSEBuildingID', 'DataYear', 'BuildingType',
       'PropertyName','Neighborhood','YearBuilt','NumberofBuildings','NumberofFloors',
        'PropertyGFATotal','PropertyGFAParking','PropertyGFABuilding(s)',
       'LargestPropertyUseType', 'ENERGYSTARScore',
       'SiteEnergyUseWN(kBtu)', 'SteamUse(kBtu)',
        'Electricity(kBtu)','NaturalGas(kBtu)','TotalGHGEmissions']
data = data[cols]

In [292]:
data['BuildingType'].value_counts()

NonResidential          1460
Multifamily LR (1-4)    1018
Multifamily MR (5-9)     580
Multifamily HR (10+)     110
SPS-District K-12         98
Nonresidential COS        85
Campus                    24
Nonresidential WA          1
Name: BuildingType, dtype: int64

In [293]:
data_length_before = data.shape[0]
data['BuildingType'].replace({'Nonresidential COS': 'NonResidential',
                              'Nonresidential WA': 'NonResidential'}, inplace=True)
data = data.loc[data['BuildingType'] == 'NonResidential']

print(f'lignes supprimés  {data_length_before - data.shape[0]}')

lignes supprimés  1830


In [294]:
#valeurs manquantes
data.isna().sum()

OSEBuildingID               0
DataYear                    0
BuildingType                0
PropertyName                0
Neighborhood                0
YearBuilt                   0
NumberofBuildings           2
NumberofFloors              0
PropertyGFATotal            0
PropertyGFAParking          0
PropertyGFABuilding(s)      0
LargestPropertyUseType      6
ENERGYSTARScore           540
SiteEnergyUseWN(kBtu)       3
SteamUse(kBtu)              2
Electricity(kBtu)           2
NaturalGas(kBtu)            2
TotalGHGEmissions           2
dtype: int64

In [295]:
#drop toutes lignes qui ont des valeurs nulles sauf EnergyScore
data_length_before = data.shape[0]
cols = ['LargestPropertyUseType', 'SiteEnergyUseWN(kBtu)', 'TotalGHGEmissions', 'NaturalGas(kBtu)', 'Electricity(kBtu)', 'SteamUse(kBtu)', 'NumberofBuildings']
data.dropna(subset=cols, inplace=True)
data.isna().sum()
print(f'lignes supprimés  {data_length_before - data.shape[0]}')

lignes supprimés  7


In [296]:
data.describe()

,OSEBuildingID,DataYear,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),ENERGYSTARScore,SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kBtu),NaturalGas(kBtu),TotalGHGEmissions
count,1539.000000,1539.0,1539.000000,1539.000000,1539.000000,1.539000e+03,1539.000000,1.539000e+03,1003.000000,1.539000e+03,1.539000e+03,1.539000e+03,1.539000e+03,1539.000000
mean,16609.779727,2016.0,1961.307342,1.035737,4.283951,1.120106e+05,13829.220273,9.818137e+04,63.790628,7.761522e+06,4.967574e+05,5.435044e+06,1.672264e+06,165.047498
std,13858.481096,0.0,32.888052,0.569622,6.796834,1.918960e+05,43730.728115,1.685664e+05,28.923639,1.884349e+07,5.314667e+06,1.305847e+07,5.513280e+06,571.657895
min,1.000000,2016.0,1900.000000,0.000000,0.000000,1.128500e+04,0.000000,3.636000e+03,1.000000,0.000000e+00,0.000000e+00,-1.154170e+05,0.000000e+00,-0.800000
25%,609.500000,2016.0,1929.000000,1.000000,1.000000,2.872800e+04,0.000000,2.768500e+04,44.000000,1.265929e+06,0.000000e+00,7.075250e+05,0.000000e+00,19.335000
50%,21213.000000,2016.0,1965.000000,1.000000,2.000000,4.721500e+04,0.000000,4.508200e+04,71.000000,2.739122e+06,0.000000e+00,1.633105e+06,4.313920e+05,48.350000
75%,24629.500000,2016.0,1988.000000,1.000000,4.000000,1.028880e+05,0.000000,9.290600e+04,89.000000,7.227405e+06,0.000000e+00,5.046944e+06,1.409886e+06,136.735000
max,50226.000000,2016.0,2015.000000,9.000000,99.000000,1.952220e+06,512608.000000,1.765970e+06,100.000000,2.966717e+08,1.349435e+08,2.745325e+08,1.381912e+08,12307.160000


In [297]:
data_length_before = data.shape[0]
cols = data.select_dtypes(include=[np.number]).columns
data = data[~(data[cols] < 0).any(axis=1)].copy()
print(f'lignes supprimés  {data_length_before - data.shape[0]}')

lignes supprimés  1


In [298]:
data_length_before = data.shape[0]
difference = abs(data['PropertyGFATotal'] - (data['PropertyGFABuilding(s)'] + data['PropertyGFAParking']))
data = data[difference == 0]
print(f'lignes supprimés  {data_length_before - data.shape[0]}')

lignes supprimés  0


In [299]:
#création d'une colonne ancienneté à partir des variable DataYear et YearBuilt
data['Ancienneté'] = data['DataYear'] - data['YearBuilt']
data = data.drop('YearBuilt', axis=1)


In [300]:
# Création de nouvelles colonnes binaires pour NumberofFloors et NumberofBuildings
floors_dummies = pd.get_dummies(data['NumberofFloors'], prefix='Floors')
buildings_dummies = pd.get_dummies(data['NumberofBuildings'], prefix='Buildings')
data = pd.concat([data, floors_dummies, buildings_dummies], axis=1)

# Suppression des colonnes binaires contenant seulement des zéros
zero_columns = [col for col in data.columns if all(data[col] == 0)]
data = data.drop(zero_columns, axis=1)

In [301]:
numeric_cols = ['NumberofFloors','NumberofBuildings','PropertyGFATotal', 'PropertyGFAParking', 'PropertyGFABuilding(s)', 'ENERGYSTARScore', 'SiteEnergyUseWN(kBtu)', 'SteamUse(kBtu)', 'Electricity(kBtu)', 'NaturalGas(kBtu)', 'TotalGHGEmissions', 'Ancienneté']
df_numeric = data[numeric_cols]
scaler = StandardScaler()
df_numeric_scaled = pd.DataFrame(scaler.fit_transform(df_numeric), columns=df_numeric.columns)
data[numeric_cols] = df_numeric_scaled

In [302]:
#energy non nulle
data_energy_star_filtered = data[data['ENERGYSTARScore'].notna()]
data = data[data['ENERGYSTARScore'].notna()]

In [303]:
data.to_csv('cleaned_data.csv', index=False)

In [305]:
data.shape[0]

621

In [306]:
data.columns

Index(['OSEBuildingID', 'DataYear', 'BuildingType', 'PropertyName',
       'Neighborhood', 'NumberofBuildings', 'NumberofFloors',
       'PropertyGFATotal', 'PropertyGFAParking', 'PropertyGFABuilding(s)',
       'LargestPropertyUseType', 'ENERGYSTARScore', 'SiteEnergyUseWN(kBtu)',
       'SteamUse(kBtu)', 'Electricity(kBtu)', 'NaturalGas(kBtu)',
       'TotalGHGEmissions', 'Ancienneté', 'Floors_0', 'Floors_1', 'Floors_2',
       'Floors_3', 'Floors_4', 'Floors_5', 'Floors_6', 'Floors_7', 'Floors_8',
       'Floors_9', 'Floors_10', 'Floors_11', 'Floors_12', 'Floors_13',
       'Floors_14', 'Floors_15', 'Floors_16', 'Floors_17', 'Floors_18',
       'Floors_19', 'Floors_20', 'Floors_21', 'Floors_22', 'Floors_23',
       'Floors_24', 'Floors_25', 'Floors_27', 'Floors_28', 'Floors_29',
       'Floors_30', 'Floors_33', 'Floors_34', 'Floors_36', 'Floors_37',
       'Floors_39', 'Floors_41', 'Floors_42', 'Floors_46', 'Floors_47',
       'Floors_49', 'Floors_55', 'Floors_56', 'Floors_63', 'Floo